### Build a basic chatbot with langgraph using graph api

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()
llm = init_chat_model("claude-sonnet-4-5-20250929")

In [ ]:
# Node functionality
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
graph_builder = StateGraph(State)

# Adding nodes
graph_builder.add_node("chatbot", chatbot)

# Adding edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Compile the graph
graph = graph_builder.compile()

In [ ]:
# Visualize the graph
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print("Graph visualization failed:", e)

In [ ]:
# Visualize the graph - simple version
graph

In [ ]:
response = graph.invoke({"messages": "Hello"})

In [ ]:
response["messages"]

In [ ]:
response["messages"][-1].content

In [ ]:
for event in graph.stream({"messages": "Hi, how are you?"}):
    for value in event.values():
        print(value["messages"][-1].content)

### Chatbot with tools

In [ ]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=2)
tool.invoke("What is the weather in New York?")

In [ ]:
# Custom tool function
def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the result.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of the two integers.
    """
    return a * b

In [ ]:
tools = [tool, multiply]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm_with_tools

In [ ]:
# Stagegraph
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

# Node definition
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Graph
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools=tools))

# Add edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm", 
    # if the latest message (result) from assistant is a tool call -> tools_condition routes to tools node
    # if the latest messsage (result) from assistant is not a tool call -> tools_condition routes to END
    tools_condition
    )
builder.add_edge("tools", END)

# Compile the graph
graph = builder.compile()

graph

In [ ]:
response = graph.invoke({"messages": "What is the recent AI news?"})
response["messages"][-1].content

In [ ]:
for m in response["messages"]:
    m.pretty_print()

In [ ]:
response = graph.invoke({"messages": "What is the result of multiplying 7 and 8 then multiplying the result by 2?"})

for m in response["messages"]:
    m.pretty_print()

### ReAct Agent Architecture

In [ ]:
# Stagegraph
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

# Node definition
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Graph
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools=tools))

# Add edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm", 
    # if the latest message (result) from assistant is a tool call -> tools_condition routes to tools node
    # if the latest messsage (result) from assistant is not a tool call -> tools_condition routes to END
    tools_condition
    )
builder.add_edge("tools", "tool_calling_llm")

# Compile the graph
graph = builder.compile()

graph

In [ ]:
response = graph.invoke({"messages": "Give me the recent AI news and the result of multiplying 6 and 9."})

for m in response["messages"]:
    m.pretty_print()

### Add Memory to Agentic Graph

In [ ]:
# Stagegraph
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# create memory object
memory = MemorySaver()

# Node definition
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Graph
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools=tools))

# Add edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm", 
    # if the latest message (result) from assistant is a tool call -> tools_condition routes to tools node
    # if the latest messsage (result) from assistant is not a tool call -> tools_condition routes to END
    tools_condition
    )
builder.add_edge("tools", "tool_calling_llm")

# Compile the graph
graph = builder.compile(checkpointer=memory)

graph

In [ ]:
config = {"configurable": {"thread_id": "1"}}

response = graph.invoke({"messages": "Hi, my name is Huy"}, config=config)

for m in response["messages"]:
    m.pretty_print()

In [ ]:
response = graph.invoke({"messages": "Hey, what is my name?"}, config=config)

for m in response["messages"]:
    m.pretty_print()

### Streaming

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [ ]:
def superbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
graph = StateGraph(State)

# nodes
graph.add_node("superbot", superbot)

# edges
graph.add_edge(START, "superbot")
graph.add_edge("superbot", END)

# compile
graph_builder = graph.compile(checkpointer=memory)

# visualize
graph_builder

In [ ]:
# invocation
config = {"configurable": {"thread_id": "1"}}

graph_builder.invoke({"messages": "Hello"}, config=config)